In [11]:
import pandas as pd
import numpy as np
import re
import warnings
from fancyimpute import IterativeImputer
import os
import jieba
from snownlp import SnowNLP
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Lasso
from statsmodels.stats.outliers_influence import variance_inflation_factor
warnings.filterwarnings("ignore", message="Skipping features without any observed values")


## 处理Price数据

In [44]:
#文本处理函数
# 文本预处理：去除特殊字符、多余空格
def clean_text(text):
    if pd.isna(text) or text == '':
        return ''
    text = str(text).strip()
    # 保留中英文和数字，其他字符替换为空格
    text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9]", " ", text)
    # 合并连续空格
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 中文分词（使用jieba）
def segment_text(text):
    if not text:
        return ''
    # 分词并过滤长度<=1的无意义词
    words = [w for w in jieba.cut(text) if len(w) > 1]
    return " ".join(words)

# 情感分析（基于SnowNLP，输出情感得分和倾向）
def analyze_sentiment(text):
    if not text:
        return 0.5, "中性"  # 空文本默认中性
    s = SnowNLP(text)
    score = round(s.sentiments, 3)  # 0-1，越接近1越积极
    if score > 0.6:
        return score, "积极"
    elif score < 0.4:
        return score, "消极"
    else:
        return score, "中性"

# 文本特征提取（词袋模型，保留高频关键词）
def extract_text_features(text_series, top_k=50):
    """将分词后的文本转换为数值特征（词频向量）"""
    # 初始化词袋模型，保留top_k个高频词
    vectorizer = CountVectorizer(max_features=top_k)
    # 拟合并转换文本数据
    text_matrix = vectorizer.fit_transform(text_series)
    # 转换为DataFrame，列名为关键词
    feature_cols = [f"反馈关键词_{word}" for word in vectorizer.get_feature_names_out()]
    text_features = pd.DataFrame(
        text_matrix.toarray(),
        columns=feature_cols,
        index=text_series.index
    )
    return text_features, vectorizer

# 根据经纬度确定环线的函数
def get_ring_road(coord_x, coord_y):
    if 116.35 <= coord_x <= 116.45 and 39.9 <= coord_y <= 40.05:
        return '二环内'
    elif 116.25 <= coord_x <= 116.5 and 39.85 <= coord_y <= 40.1:
        return '二至三环'
    elif 116.1 <= coord_x <= 116.6 and 39.7 <= coord_y <= 40.2:
        return '三至四环'
    elif 116.0 <= coord_x <= 116.7 and 39.6 <= coord_y <= 40.3:
        return '四至五环'
    elif 115.9 <= coord_x <= 116.8 and 39.5 <= coord_y <= 40.4:
        return '五至六环'
    return '六环外'

# 通用范围值处理函数（提取数值，范围取平均）
def process_range_fee(fee_str, unit):
    if pd.isnull(fee_str) or fee_str == '':
        return np.nan
    fee_str = str(fee_str).strip().replace(unit, '').strip()
    if '-' in fee_str:
        parts = [p.strip() for p in fee_str.split('-') if p.strip()]
        if len(parts) == 2:
            try:
                return (float(parts[0]) + float(parts[1])) / 2
            except ValueError:
                return np.nan
    try:
        return float(fee_str)
    except ValueError:
        return np.nan


# 众数填充函数（覆盖所有类别列，除了环线和配备电梯）
def fill_categorical_mode_except_ring(df):
    cat_cols = [col for col in df.select_dtypes(include='category').columns 
                if col not in ['环线', '配备电梯'] and col in df.columns]  # 排除配备电梯
    for col in cat_cols:
        mode_val = df[col].mode().iloc[0] if not df[col].mode().empty else None
        if mode_val is not None:
            df[col] = df[col].fillna(mode_val)
    return df

# 1. 房屋户型特殊处理：提取居室、厅、卫数量并编码
def process_house_layout(df):
    if '房屋户型' not in df.columns:
        return df
    # 提取居室、厅、卫数量
    df['居室数'] = df['房屋户型'].str.extract(r'(\d+)室').astype(float, errors='ignore').fillna(0)
    df['厅数'] = df['房屋户型'].str.extract(r'(\d+)厅').astype(float, errors='ignore').fillna(0)
    df['卫数'] = df['房屋户型'].str.extract(r'(\d+)卫').astype(float, errors='ignore').fillna(0)
    # 简化户型（如“2室1厅1卫”），异常值归为“其他”
    df['简化户型'] = df.apply(
        lambda row: f"{int(row['居室数'])}室{int(row['厅数'])}厅{int(row['卫数'])}卫" 
        if row['居室数'] > 0 else '其他', 
        axis=1
    )
    # 对简化户型进行One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    layout_dummies = encoder.fit_transform(df[['简化户型']])
    layout_df = pd.DataFrame(
        layout_dummies, 
        columns=encoder.get_feature_names_out(['简化户型']),
        index=df.index
    )
    # 合并并删除原始列
    df = pd.concat([df, layout_df], axis=1)
    df = df.drop(['房屋户型', '居室数', '厅数', '卫数', '简化户型'], axis=1, errors='ignore')
    return df


# 2. 房屋朝向特殊处理：保留具体朝向并编码
def process_orientation(df):
    if '房屋朝向' not in df.columns:
        return df
    # 标准化朝向表述
    df['房屋朝向'] = df['房屋朝向'].str.strip().replace(
        {'南北通透': '南北', '东西向': '东西', '东南向': '东南', '西南向': '西南', 
         '东北向': '东北', '西北向': '西北', '南向': '南', '北向': '北', '东向': '东', '西向': '西'}
    )
    # 定义主要朝向类别（保留具体值，控制基数）
    main_orientations = ['南北', '南', '北', '东', '西', '东南', '西南', '东北', '西北', '东西', '未知']
    df['具体朝向'] = df['房屋朝向'].apply(
        lambda x: x if x in main_orientations else '其他组合'
    )
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    ori_dummies = encoder.fit_transform(df[['具体朝向']])
    ori_df = pd.DataFrame(
        ori_dummies, 
        columns=encoder.get_feature_names_out(['具体朝向']),
        index=df.index
    )
    # 合并并删除原始列
    df = pd.concat([df, ori_df], axis=1)
    df = df.drop(['房屋朝向', '具体朝向'], axis=1, errors='ignore')
    return df


# 3. 梯户比例特殊处理：转换为数值型梯户比
def process_lift_house_ratio(df):
    if '梯户比例' not in df.columns:
        return df
    # 提取梯数和户数（缺失时合理填充）
    df['梯数'] = df['梯户比例'].str.extract(r'(\d+)梯').astype(float, errors='ignore').fillna(1)  # 默认1梯
    df['户数'] = df['梯户比例'].str.extract(r'(\d+)户').astype(float, errors='ignore').fillna(df['梯数'] * 3)  # 默认1梯3户
    # 计算梯户比（数值越小，居住密度越低）
    df['梯户比'] = df['户数'] / df['梯数']
    # 删除中间列和原始列
    df = df.drop(['梯户比例', '梯数', '户数'], axis=1, errors='ignore')
    return df


# 4. 产权描述特殊处理：合并核心产权类型并编码
def process_property_right(df):
    if '产权描述' not in df.columns:
        return df
    # 扩展类别集合，加入'其他'
    df['产权描述'] = df['产权描述'].cat.add_categories(['其他'])
    # 核心产权类型（按优先级排序）
    core_types = ['商品房', '已购公房', '一类经济适用房', '二类经济适用房', '央产房', '私产', '使用权']
    # 提取核心类型，多类型用“|”连接，无匹配则为“其他”
    df['简化产权'] = df['产权描述'].apply(
        lambda x: '|'.join([t for t in core_types if t in str(x)]) if pd.notnull(x) else '其他'
    )
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    prop_dummies = encoder.fit_transform(df[['简化产权']])
    prop_df = pd.DataFrame(
        prop_dummies, 
        columns=encoder.get_feature_names_out(['简化产权']),
        index=df.index
    )
    # 合并并删除原始列
    df = pd.concat([df, prop_df], axis=1)
    df = df.drop(['产权描述', '简化产权'], axis=1, errors='ignore')
    return df


# 5. 物业类别特殊处理：合并为核心类别并编码
def process_property_type(df):
    if '物业类别' not in df.columns:
        return df
    # 简化物业类型
    def simplify_type(typ):
        if pd.isnull(typ):
            return '其他'
        typ = typ.strip()
        if '普通住宅' in typ:
            return '普通住宅'
        elif '别墅' in typ:
            return '别墅'
        elif '商业' in typ or '写字楼' in typ:
            return '商业/写字楼'
        elif '车库' in typ or '车位' in typ:
            return '车库/车位'
        else:
            return '其他'
    df['简化物业类型'] = df['物业类别'].apply(simplify_type)
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    type_dummies = encoder.fit_transform(df[['简化物业类型']])
    type_df = pd.DataFrame(
        type_dummies, 
        columns=encoder.get_feature_names_out(['简化物业类型']),
        index=df.index
    )
    # 合并并删除原始列
    df = pd.concat([df, type_df], axis=1)
    df = df.drop(['物业类别', '简化物业类型'], axis=1, errors='ignore')
    return df


# 6. 配备电梯缺失值处理：总楼层>6则填充为“有”
def process_elevator(df):
    if '配备电梯' not in df.columns or '总楼层' not in df.columns:
        return df
    df.loc[(df['总楼层'] > 6) & df['配备电梯'].isnull(), '配备电梯'] = '有'
    # 转换为类别型并编码
    df['配备电梯'] = df['配备电梯'].astype('category')
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    elevator_dummies = encoder.fit_transform(df[['配备电梯']])
    elevator_df = pd.DataFrame(
        elevator_dummies, 
        columns=encoder.get_feature_names_out(['配备电梯']),
        index=df.index
    )
    df = pd.concat([df, elevator_df], axis=1).drop('配备电梯', axis=1)
    return df


# 类别特征编码处理函数（处理剩余低基数类别）
def encode_categorical_features(df):
    # 分离需要编码的类别列（排除已处理的环线和datetime类型）
    cat_cols = [col for col in df.select_dtypes(include='category').columns 
                if col not in df.select_dtypes(include=['datetime64']).columns]
    
    # 对低基数类别列进行One-Hot编码（基数太高的列暂不编码）
    encoded_dfs = []
    high_cardinality_cols = []
    
    for col in cat_cols:
        # 计算类别基数
        cardinality = df[col].nunique()
        # 基数小于50的列进行One-Hot编码
        if cardinality > 1 and cardinality <= 50:
            encoder = OneHotEncoder(sparse_output=False, drop='first')
            encoded = encoder.fit_transform(df[[col]])
            encoded_df = pd.DataFrame(
                encoded, 
                columns=encoder.get_feature_names_out([col]),
                index=df.index
            )
            encoded_dfs.append(encoded_df)
        else:
            high_cardinality_cols.append(col)
    
    # 合并编码后的特征
    if encoded_dfs:
        encoded_features = pd.concat(encoded_dfs, axis=1)
        df = pd.concat([df, encoded_features], axis=1)
        # 删除原始类别列
        df = df.drop(columns=cat_cols)
    if high_cardinality_cols:
        print(f"高基数类别列（未编码）：{high_cardinality_cols}")
    
    return df


# 预处理函数
def preprocess_price_file(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    columns_to_drop = ['别墅类型', '物业办公电话','抵押信息']
    df = df.drop(columns=columns_to_drop, errors='ignore')  
    
    # 数据类型转换
    cat_cols = [ '房屋朝向', '交易时间', '产权描述','物业类别', '开发商','环线',
                '配备电梯', '梯户比例', '房屋户型', '装修情况','建筑结构','交易权属',
                '上次交易', '房屋用途', '房屋年限', '产权所属','供水', '供暖', '供电','房屋优势']  
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')
           

    # 处理部分数据类型列
    if '建筑面积' in df.columns:
        df['建筑面积'] = df['建筑面积'].str.replace('㎡', '').astype(float)
    if '套内面积' in df.columns:
        df['套内面积'] = df['套内面积'].str.replace('㎡', '').astype(float)
    if '房屋总数' in df.columns:
        df['房屋总数'] = df['房屋总数'].str.extract(r'(\d+)').astype(float, errors='ignore')
    if '楼栋总数' in df.columns:
        df['楼栋总数'] = df['楼栋总数'].str.extract(r'(\d+)').astype(float, errors='ignore')
    if '物 业 费' in df.columns:
        def process_property_fee(fee):
            if pd.isnull(fee):
                return np.nan
            fee = str(fee).strip()
            nums = [float(num) for num in re.findall(r'\d+\.\d+|\d+', fee)]
            if len(nums) == 0:
                return np.nan
            elif len(nums) == 1:
                return nums[0]
            else:
                return sum(nums) / len(nums)
        df['物 业 费'] = df['物 业 费'].apply(process_property_fee).astype(float)
    if '燃气费' in df.columns:
        df['燃气费'] = df['燃气费'].apply(lambda x: process_range_fee(x, '元/m³')).astype(float)
    if '建筑年代' in df.columns:
        df['建筑年代'] = df['建筑年代'].apply(lambda x: process_range_fee(x, '年')).astype(float)
    if '供热费' in df.columns:
        df['供热费'] = df['供热费'].apply(lambda x: process_range_fee(x, '元/㎡')).astype(float)
    if '停车费用' in df.columns:
        def clean_parking_fee(fee):
            if pd.isnull(fee):
                return np.nan
            fee = str(fee).strip()
            if fee == '暂无':
                return np.nan
            elif '免费' in fee:
                return 0.0
            nums = [float(num) for num in re.findall(r'\d+\.?\d*', fee)]
            if len(nums) == 0:
                return np.nan
            elif len(nums) == 1:
                return nums[0]
            else:
                return sum(nums) / len(nums)
        df['停车费用'] = df['停车费用'].apply(clean_parking_fee).astype(float)
    #数值列插补
    numeric_df = df.select_dtypes(include=['number']).dropna(axis=1, how='all').reset_index(drop=True)
    if not numeric_df.empty:
        imputer = IterativeImputer(random_state=111)
        imputed_numeric_array = imputer.fit_transform(numeric_df)
        imputed_numeric_df = pd.DataFrame(imputed_numeric_array, columns=numeric_df.columns, index=numeric_df.index)
        # 将插补后的结果合并回原始df
        df[imputed_numeric_df.columns] = imputed_numeric_df 
        print(f"数值列插补涉及的列：{numeric_df.columns.tolist()}")
    else:
        imputed_numeric_df = numeric_df
        print(f"数值列插补涉及的列：{numeric_df.columns.tolist()}")

    #众数填充
    df = fill_categorical_mode_except_ring(df)
    
    # 环线列单独处理 
    if '环线' in df.columns and 'coord_x' in df.columns and 'coord_y' in df.columns:
        df['环线位置'] = df.apply(
    lambda row: get_ring_road(row['coord_x'], row['coord_y']) 
    if pd.isna(row['环线位置'])  # 只处理缺失值
    else row['环线位置'], 
    axis=1)
    

    # 处理交易时间特征（提取年份和月份）
    if '交易时间' in df.columns:
        df['交易时间'] = pd.to_datetime(df['交易时间'], errors='coerce')
        df['交易年份'] = df['交易时间'].dt.year
        df['交易月份'] = df['交易时间'].dt.month
        df = df.drop('交易时间', axis=1)

    if '上次交易' in df.columns:
        df['上次交易'] = pd.to_datetime(df['上次交易'], errors='coerce')
        df['上次交易年份'] = df['上次交易'].dt.year
        df['上次交易月份'] = df['上次交易'].dt.month
        df = df.drop('上次交易', axis=1)

    # 核心高基数类别特殊处理（按顺序调用）
    df = process_house_layout(df)          # 房屋户型
    df = process_orientation(df)           # 房屋朝向
    df = process_lift_house_ratio(df)      # 梯户比例
    df = process_property_right(df)        # 产权描述
    df = process_property_type(df)         # 物业类别
    # 对剩余类别特征进行编码处理
    df = encode_categorical_features(df) 
    
    # 所在楼层处理（需在配备电梯处理前完成，因为依赖总楼层）
    if '所在楼层' in df.columns:
        df['实际楼层'] = df['所在楼层'].str.extract(r'^(.*?)\s*\(')
        df['总楼层'] = df['所在楼层'].str.extract(r'共(\d+)层').astype(float)
        # 对实际楼层编码
        if not df['实际楼层'].isna().all():
            floor_encoder = OneHotEncoder(sparse_output=False, drop='first')
            floor_encoded = floor_encoder.fit_transform(df[['实际楼层']])
            floor_df = pd.DataFrame(
                floor_encoded,
                columns=floor_encoder.get_feature_names_out(['实际楼层']),
                index=df.index
            )
            df = pd.concat([df, floor_df], axis=1)
        df.drop(['所在楼层', '实际楼层'], axis=1, inplace=True, errors='ignore')

    # 配备电梯缺失值处理
    df = process_elevator(df)

    #异常值处理
    if 'Price' in df.columns:
        Q1, Q3 = df['Price'].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        df = df[(df['Price'] >= Q1 - 1.5*IQR) & (df['Price'] <= Q3 + 1.5*IQR)]
        print(f"文件{file_path}：Price列异常值处理完成，保留{len(df)}行数据")


    # 新增：处理“客户反馈”列文本
    if '客户反馈' in df.columns:
        print(f"正在处理{len(df)}条客户反馈文本...")
        
        # 1. 文本清洗
        df['反馈_清洗后'] = df['客户反馈'].apply(clean_text)
        
        # 2. 中文分词
        df['反馈_分词后'] = df['反馈_清洗后'].apply(segment_text)
        
        # 3. 情感分析
        sentiment_results = df['反馈_清洗后'].apply(analyze_sentiment)
        df['反馈_情感得分'] = [res[0] for res in sentiment_results]
        df['反馈_情感倾向'] = [res[1] for res in sentiment_results]
        
        # 4. 提取文本数值特征（词袋模型）
        # 过滤空分词结果
        valid_texts = df['反馈_分词后'].replace('', np.nan).dropna()
        if len(valid_texts) > 0:
            text_features, _ = extract_text_features(valid_texts, top_k=50)
            # 合并文本特征到主数据框（空文本对应特征值全为0）
            df = pd.concat([df, text_features.reindex(df.index).fillna(0)], axis=1)
        
        # 5. 情感倾向One-Hot编码
        sentiment_encoder = OneHotEncoder(sparse_output=False, drop='first')
        sentiment_dummies = sentiment_encoder.fit_transform(df[['反馈_情感倾向']])
        sentiment_df = pd.DataFrame(
            sentiment_dummies,
            columns=[f"反馈情感_{label}" for label in sentiment_encoder.get_feature_names_out(['反馈_情感倾向'])],
            index=df.index
        )
        df = pd.concat([df, sentiment_df], axis=1)
        
        # 6. 删除中间文本列（保留数值特征和情感特征）
        df = df.drop(['客户反馈', '反馈_清洗后', '反馈_分词后', '反馈_情感倾向'], axis=1, errors='ignore')
        print("客户反馈文本处理完成，新增情感特征和关键词特征")

    df = df.drop_duplicates()

    return df


# 执行处理
base_path = '/Users/guodehao/Documents'
price_files = [
    os.path.join(base_path, 'train_price.csv'),
    os.path.join(base_path, 'test_price.csv')
]

print("===== 开始处理价格相关文件 =====")
for file_path in price_files:
    if os.path.exists(file_path): 
        processed_df = preprocess_price_file(file_path)
        file_name = os.path.basename(file_path)
        save_path = os.path.join(base_path, f'{file_name.split(".")[0]}_preprocessed.csv')
        processed_df.to_csv(save_path, index=False)
        print(f"价格文件处理完成：{save_path}\n")
    else:
        print(f"价格文件不存在：{file_path}\n")

===== 开始处理价格相关文件 =====
数值列插补涉及的列：['城市', '区域', '板块', 'Price', '建筑面积', '套内面积', 'lon', 'lat', '年份', '区县', '板块_comm', '建筑年代', '房屋总数', '楼栋总数', '容 积 率', '物 业 费', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']
高基数类别列（未编码）：['开发商']
文件/Users/guodehao/Documents/train_price.csv：Price列异常值处理完成，保留96048行数据
正在处理96048条客户反馈文本...
客户反馈文本处理完成，新增情感特征和关键词特征
价格文件处理完成：/Users/guodehao/Documents/train_price_preprocessed.csv

数值列插补涉及的列：['ID', '城市', '区域', '板块', '建筑面积', '套内面积', 'lon', 'lat', '年份', '区县', '板块_comm', '建筑年代', '房屋总数', '楼栋总数', '容 积 率', '物 业 费', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']
高基数类别列（未编码）：['开发商']
正在处理34017条客户反馈文本...
客户反馈文本处理完成，新增情感特征和关键词特征
价格文件处理完成：/Users/guodehao/Documents/test_price_preprocessed.csv



## 处理Rent数据

In [46]:
#函数修改
# 众数填充函数（覆盖所有类别列，除了环线、电梯和配套设施）
def fill_categorical_mode_except_ring(df):
    cat_cols = [col for col in df.select_dtypes(include='category').columns 
                if col not in ['环线位置', '电梯', '配套设施'] and col in df.columns]  # 排除特殊处理列
    for col in cat_cols:
        mode_val = df[col].mode().iloc[0] if not df[col].mode().empty else None
        if mode_val is not None:
            df[col] = df[col].fillna(mode_val)
    return df

# 租期处理：先按类别用众数填充缺失值，再转换为数值分数
def process_lease_term(df):
    if '租期' not in df.columns:
        return df
    if not isinstance(df['租期'].dtype, pd.CategoricalDtype):
        df['租期'] = df['租期'].astype('category')
    if df['租期'].isnull().any():
        mode_val = df['租期'].mode().iloc[0] if not df['租期'].mode().empty else '未知'
    term_score_map = {
        '1年': 12,
        '5~12个月': 8,
        '1~2年': 18,
        '1年以内': 6,
        '3年以内': 18,}
    #转换为数值分数（确保所有类别都被映射）
    # 对未定义的类别，默认赋予0分
    df['租期_分数'] = df['租期'].apply(lambda x: term_score_map.get(x, 0)).astype(int)
    df.drop('租期', axis=1, inplace=True)
    return df

# 朝向特殊处理：保留具体朝向，标准化后编码
def process_orientation(df):
    if '朝向' not in df.columns:
        return df
    # 定义所有需要的类别
    all_cats = ['南北', '南', '北', '东', '西', '东南', '西南', '东北', '西北', '东西', '未知', '其他组合']
    # 获取当前已有类别
    existing_cats = df['朝向'].cat.categories.tolist() if not df['朝向'].cat.categories.empty else []
    # 筛选完全新增的类别
    new_cats = [cat for cat in all_cats if cat not in existing_cats]
    # 扩展类别
    df['朝向'] = df['朝向'].cat.add_categories(new_cats)
    # 标准化朝向表述
    df['朝向'] = df['朝向'].str.strip().replace(
        {'南北通透': '南北', '东西向': '东西', '东南向': '东南', '西南向': '西南', 
         '东北向': '东北', '西北向': '西北', '南向': '南', '北向': '北', '东向': '东', '西向': '西'}
    )
    # 定义主要朝向类别（保留具体值）
    main_orientations = ['南北', '南', '北', '东', '西', '东南', '西南', '东北', '西北', '东西', '未知']
    df['具体朝向'] = df['朝向'].apply(lambda x: x if x in main_orientations else '其他组合')
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    ori_dummies = encoder.fit_transform(df[['具体朝向']])
    ori_df = pd.DataFrame(
        ori_dummies, 
        columns=encoder.get_feature_names_out(['具体朝向']),
        index=df.index
    )
    df = pd.concat([df, ori_df], axis=1).drop(['朝向', '具体朝向'], axis=1, errors='ignore')
    return df


# 交易时间特殊处理：提取时间特征
def process_trade_time(df):
    if '交易时间' not in df.columns:
        return df
    # 转换为datetime类型（指定格式避免警告）
    df['交易时间'] = pd.to_datetime(df['交易时间'], format='%Y-%m-%d', errors='coerce')
    # 提取年份、月份、季节特征
    df['交易年份'] = df['交易时间'].dt.year
    df['交易月份'] = df['交易时间'].dt.month
    df['交易季节'] = df['交易时间'].dt.quarter
    df.drop('交易时间', axis=1, inplace=True)
    return df


# 物业类别特殊处理：合并核心类别并编码
def process_property_type(df):
    if '物业类别' not in df.columns:
        return df
    # 定义所有需要的类别
    all_cats = ['普通住宅', '别墅', '商业/写字楼', '车库/车位', '其他']
    # 获取当前已有类别
    existing_cats = df['物业类别'].cat.categories.tolist() if not df['物业类别'].cat.categories.empty else []
    # 筛选完全新增的类别
    new_cats = [cat for cat in all_cats if cat not in existing_cats]
    # 扩展类别
    df['物业类别'] = df['物业类别'].cat.add_categories(new_cats)
    # 简化物业类型
    def simplify_type(typ):
        if pd.isnull(typ):
            return '其他'
        typ = typ.strip()
        if '普通住宅' in typ:
            return '普通住宅'
        elif '别墅' in typ:
            return '别墅'
        elif '商业' in typ or '写字楼' in typ:
            return '商业/写字楼'
        elif '车库' in typ or '车位' in typ:
            return '车库/车位'
        else:
            return '其他'
    df['简化物业类型'] = df['物业类别'].apply(simplify_type)
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    type_dummies = encoder.fit_transform(df[['简化物业类型']])
    type_df = pd.DataFrame(
        type_dummies, 
        columns=encoder.get_feature_names_out(['简化物业类型']),
        index=df.index
    )
    df = pd.concat([df, type_df], axis=1).drop(['物业类别', '简化物业类型'], axis=1, errors='ignore')
    return df


# 产权描述特殊处理：合并核心产权类型并编码
def process_property_right(df):
    if '产权描述' not in df.columns:
        return df
    # 定义所有需要的类别
    all_cats = ['商品房', '已购公房', '一类经济适用房', '二类经济适用房', '央产房', '私产', '使用权', '其他']
    # 获取当前已有类别
    existing_cats = df['产权描述'].cat.categories.tolist() if not df['产权描述'].cat.categories.empty else []
    # 筛选完全新增的类别
    new_cats = [cat for cat in all_cats if cat not in existing_cats]
    # 扩展类别
    df['产权描述'] = df['产权描述'].cat.add_categories(new_cats)
    # 核心产权类型
    core_types = ['商品房', '已购公房', '一类经济适用房', '二类经济适用房', '央产房', '私产', '使用权']
    # 提取核心类型，多类型用“|”连接
    df['简化产权'] = df['产权描述'].apply(
        lambda x: '|'.join([t for t in core_types if t in str(x)]) if pd.notnull(x) else '其他'
    )
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    prop_dummies = encoder.fit_transform(df[['简化产权']])
    prop_df = pd.DataFrame(
        prop_dummies, 
        columns=encoder.get_feature_names_out(['简化产权']),
        index=df.index
    )
    df = pd.concat([df, prop_df], axis=1).drop(['产权描述', '简化产权'], axis=1, errors='ignore')
    return df


# 电梯缺失值处理：总楼层>6则填充为“有”
def process_elevator(df):
    if '电梯' not in df.columns or '总楼层' not in df.columns:
        return df
    # 定义所有需要的类别
    all_cats = ['有', '无']
    # 获取当前已有类别
    existing_cats = df['电梯'].cat.categories.tolist() if not df['电梯'].cat.categories.empty else []
    # 筛选完全新增的类别
    new_cats = [cat for cat in all_cats if cat not in existing_cats]
    # 扩展类别
    df['电梯'] = df['电梯'].cat.add_categories(new_cats)
    # 按规则填充缺失值
    df.loc[(df['总楼层'] > 6) & df['电梯'].isnull(), '电梯'] = '有'
    # 剩余缺失值填充为'无'
    df['电梯'] = df['电梯'].fillna('无')
    # One-Hot编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    elevator_dummies = encoder.fit_transform(df[['电梯']])
    elevator_df = pd.DataFrame(
        elevator_dummies, 
        columns=encoder.get_feature_names_out(['电梯']),
        index=df.index
    )
    df = pd.concat([df, elevator_df], axis=1).drop('电梯', axis=1)
    return df


# 配套设施处理函数（按设施数量计分）
def process_amenities(df):
    if '配套设施' not in df.columns:
        return df
    # 先将类别列转换为字符串类型（避免Categorical类型限制）
    df['配套设施'] = df['配套设施'].astype(str)
    # 处理缺失值（视为0个设施）
    df['配套设施'] = df['配套设施'].fillna('')
    # 按分隔符拆分并计数（支持“、”“，”“;”等常见分隔符）
    df['配套设施得分'] = df['配套设施'].apply(
        lambda x: len([item for item in re.split(r'[、,;]', str(x)) if item.strip()])
    )
    # 转换为整数类型
    df['配套设施得分'] = df['配套设施得分'].astype(int)
    # 删除原始列
    df.drop('配套设施', axis=1, inplace=True)
    return df


# 类别特征编码处理函数（处理剩余低基数类别）
def encode_categorical_features(df):
    # 分离需要编码的类别列（排除已处理的环线和datetime类型）
    cat_cols = [col for col in df.select_dtypes(include='category').columns 
                if col not in df.select_dtypes(include=['datetime64']).columns]
    
    # 对低基数类别列进行One-Hot编码（基数太高的列暂不编码）
    encoded_dfs = []
    high_cardinality_cols = []
    
    for col in cat_cols:
        # 计算类别基数
        cardinality = df[col].nunique()
        # 基数小于50的列进行One-Hot编码
        if cardinality > 1 and cardinality <= 50:
            encoder = OneHotEncoder(sparse_output=False, drop='first')
            encoded = encoder.fit_transform(df[[col]])
            encoded_df = pd.DataFrame(
                encoded, 
                columns=encoder.get_feature_names_out([col]),
                index=df.index
            )
            encoded_dfs.append(encoded_df)
        else:
            high_cardinality_cols.append(col)
    
    if encoded_dfs:
        encoded_features = pd.concat(encoded_dfs, axis=1)
        df = pd.concat([df, encoded_features], axis=1)
        df = df.drop(columns=cat_cols)
    print(f"未编码高基数类别列：{high_cardinality_cols}")
    return df


# 预处理函数
def preprocess_price_file(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    # 删除内容缺失过多的列
    columns_to_drop = ['别墅类型', '物业办公电话']
    df = df.drop(columns=columns_to_drop, errors='ignore')  
    
    # 数据类型转换
    cat_cols = [ '朝向', '交易时间', '付款方式', '租赁方式', '装修','租期','楼层',
                '电梯', '车位', '用水', '用电', '燃气', '采暖', '配套设施',
               '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
               '建筑结构','产权描述', '供水', '供暖', '供电']  
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')  # 确保所有类别列正确转换
            
    # 处理部分数据类型列
    # 处理面积列
    if '面积' in df.columns:
        df['面积'] = df['面积'].astype(str).str.replace('㎡', '').astype(float)
    # 处理绿化率
    if '绿 化 率' in df.columns:
        df['绿 化 率'] = df['绿 化 率'].astype(str).str.replace('%', '').astype(float)
    # 处理房屋总数和楼栋总数为数值型
    if '房屋总数' in df.columns:
        df['房屋总数'] = df['房屋总数'].str.extract(r'(\d+)').astype(float, errors='ignore')
    if '楼栋总数' in df.columns:
        df['楼栋总数'] = df['楼栋总数'].str.extract(r'(\d+)').astype(float, errors='ignore')

    # 处理物业费为数值型
    if '物 业 费' in df.columns:
        def process_property_fee(fee):
            if pd.isnull(fee):
                return np.nan
            fee = str(fee).strip()
            nums = [float(num) for num in re.findall(r'\d+\.\d+|\d+', fee)]
            if len(nums) == 0:
                return np.nan
            elif len(nums) == 1:
                return nums[0]
            else:
                return sum(nums) / len(nums)
        
        df['物 业 费'] = df['物 业 费'].apply(process_property_fee).astype(float)

    # 处理燃气费（支持范围值取平均）
    if '燃气费' in df.columns:
        df['燃气费'] = df['燃气费'].apply(lambda x: process_range_fee(x, '元/m³')).astype(float)

    # 处理供热费（支持范围值取平均）
    if '供热费' in df.columns:
        df['供热费'] = df['供热费'].apply(lambda x: process_range_fee(x, '元/㎡')).astype(float)
    if '建筑年代' in df.columns:
        df['建筑年代'] = df['建筑年代'].apply(lambda x: process_range_fee(x, '年')).astype(float)
    # 处理停车费用为数值型
    if '停车费用' in df.columns:
        def clean_parking_fee(fee):
            if pd.isnull(fee):
                return np.nan
            fee = str(fee).strip()
            if fee == '暂无':
                return np.nan
            elif '免费' in fee:
                return 0.0
            nums = [float(num) for num in re.findall(r'\d+\.?\d*', fee)]
            if len(nums) == 0:
                return np.nan
            elif len(nums) == 1:
                return nums[0]
            else:
                return sum(nums) / len(nums)
        df['停车费用'] = df['停车费用'].apply(clean_parking_fee).astype(float)
    
    # 数值列插补（确保租期_月被识别为数值特征）
    numeric_df = df.select_dtypes(include=['int', 'float']).dropna(axis=1, how='all').reset_index(drop=True)
    if not numeric_df.empty:
        imputer = IterativeImputer(random_state=111)
        imputed_numeric_array = imputer.fit_transform(numeric_df)
        imputed_numeric_df = pd.DataFrame(imputed_numeric_array, columns=numeric_df.columns, index=numeric_df.index)
    else:
        imputed_numeric_df = numeric_df
    print(f"数值列插补涉及的列：{numeric_df.columns.tolist()}")

    
    # 调用众数填充函数（跳过特殊处理列）
    df = fill_categorical_mode_except_ring(df)
    
    # 环线列单独处理 
    if '环线' in df.columns and 'coord_x' in df.columns and 'coord_y' in df.columns:
        df['环线位置'] = df.apply(
    lambda row: get_ring_road(row['coord_x'], row['coord_y']) 
    if pd.isna(row['环线位置'])  # 只处理缺失值
    else row['环线位置'], 
    axis=1)

    # 合并数据
    df = pd.concat([imputed_numeric_df, df.select_dtypes(exclude=['number'])], axis=1)

    # 异常值处理（Price列）
    if 'Price' in df.columns:
        Q1, Q3 = df['Price'].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        df = df[(df['Price'] >= Q1 - 1.5*IQR) & (df['Price'] <= Q3 + 1.5*IQR)]
        print(f"文件{file_path}：Price列异常值处理完成，保留{len(df)}行数据")

    # 楼层拆分
    if '楼层' in df.columns:
        df[['实际楼层', '总楼层']] = df['楼层'].str.split('/', expand=True)
        df['实际楼层'] = df['实际楼层'].str.extract(r'(\d+)').astype(float, errors='ignore')
        df['总楼层'] = df['总楼层'].str.extract(r'(\d+)').astype(float, errors='ignore')
        df.drop('楼层', axis=1, inplace=True)

    # 特殊编码处理（按顺序调用，新增配套设施处理）
    df = process_lease_term(df)          # 租期（已转为数值特征）
    df = process_trade_time(df)          # 交易时间
    df = process_orientation(df)         # 朝向
    df = process_property_type(df)       # 物业类别
    df = process_property_right(df)      # 产权描述
    df = process_elevator(df)            # 电梯
    df = process_amenities(df)           # 配套设施

    # 剩余类别变量编码
    df = encode_categorical_features(df)

    # 处理“客户反馈”列文本
    if '客户反馈' in df.columns:
        print(f"正在处理{len(df)}条客户反馈文本...")
        
        # 1. 文本清洗
        df['反馈_清洗后'] = df['客户反馈'].apply(clean_text)
        
        # 2. 中文分词
        df['反馈_分词后'] = df['反馈_清洗后'].apply(segment_text)
        
        # 3. 情感分析
        sentiment_results = df['反馈_清洗后'].apply(analyze_sentiment)
        df['反馈_情感得分'] = [res[0] for res in sentiment_results]
        df['反馈_情感倾向'] = [res[1] for res in sentiment_results]
        
        # 4. 提取文本数值特征（词袋模型）
        # 过滤空分词结果
        valid_texts = df['反馈_分词后'].replace('', np.nan).dropna()
        if len(valid_texts) > 0:
            text_features, _ = extract_text_features(valid_texts, top_k=50)
            # 合并文本特征到主数据框（空文本对应特征值全为0）
            df = pd.concat([df, text_features.reindex(df.index).fillna(0)], axis=1)
        
        # 5. 情感倾向One-Hot编码
        sentiment_encoder = OneHotEncoder(sparse_output=False, drop='first')
        sentiment_dummies = sentiment_encoder.fit_transform(df[['反馈_情感倾向']])
        sentiment_df = pd.DataFrame(
            sentiment_dummies,
            columns=[f"反馈情感_{label}" for label in sentiment_encoder.get_feature_names_out(['反馈_情感倾向'])],
            index=df.index
        )
        df = pd.concat([df, sentiment_df], axis=1)
        
        # 6. 删除中间文本列（保留数值特征和情感特征）
        df = df.drop(['客户反馈', '反馈_清洗后', '反馈_分词后', '反馈_情感倾向'], axis=1, errors='ignore')
        print("客户反馈文本处理完成，新增情感特征和关键词特征")
   
    
    # 去重
    df = df.drop_duplicates()

    return df


# 执行处理
base_path = '/Users/guodehao/Documents'
price_files = [
    os.path.join(base_path, 'train_rent.csv'),
    os.path.join(base_path, 'test_rent.csv')
]

print("===== 开始处理租赁相关文件 =====")
for file_path in price_files:
    if os.path.exists(file_path):  
        processed_df = preprocess_price_file(file_path)
        file_name = os.path.basename(file_path)
        save_path = os.path.join(base_path, f'{file_name.split(".")[0]}_preprocessed.csv')
        processed_df.to_csv(save_path, index=False)
        print(f"租赁文件处理完成：{save_path}\n")
    else:
        print(f"租赁文件不存在：{file_path}\n")

===== 开始处理租赁相关文件 =====
数值列插补涉及的列：['城市', 'Price', '面积', 'lon', 'lat', '年份', '区县', '板块', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']
文件/Users/guodehao/Documents/train_rent.csv：Price列异常值处理完成，保留93365行数据
未编码高基数类别列：['装修', '开发商', '物业公司']
正在处理93365条客户反馈文本...
客户反馈文本处理完成，新增情感特征和关键词特征
租赁文件处理完成：/Users/guodehao/Documents/train_rent_preprocessed.csv

数值列插补涉及的列：['ID', '城市', '面积', 'lon', 'lat', '年份', '区县', '板块', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']
未编码高基数类别列：['装修', '开发商', '物业公司']
正在处理9773条客户反馈文本...
客户反馈文本处理完成，新增情感特征和关键词特征
租赁文件处理完成：/Users/guodehao/Documents/test_rent_preprocessed.csv



## 数据划分与特征工程


In [47]:
# ---------------------- 配置路径 ----------------------
# 数据读取路径
price_data_path = "/Users/guodehao/Documents/train_price_preprocessed.csv"
rent_data_path = "/Users/guodehao/Documents/train_rent_preprocessed.csv"

# 数据保存路径（统一保存到指定文件夹）
save_path = "/Users/guodehao/Documents/python/"  # 与后续模型训练路径一致

# ---------------------- 1. 数据读取与划分 ----------------------
try:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import VarianceThreshold
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    from sklearn.linear_model import Lasso

    df_price = pd.read_csv(price_data_path, low_memory=False)
    print("房价数据读取完成，共{}条".format(len(df_price)))
    df_rent = pd.read_csv(rent_data_path, low_memory=False)
    print("租金数据读取完成，共{}条".format(len(df_rent)))

    # 划分房价数据（特征X + 标签y）
    X_price = df_price.drop('Price', axis=1)
    y_price = df_price['Price']
    X_price_train, X_price_test, y_price_train, y_price_test = train_test_split(
        X_price, y_price, test_size=0.2, random_state=111
    )
    print("房价数据划分完成：训练集{}条，测试集{}条".format(len(X_price_train), len(X_price_test)))

    # 划分租金数据（特征X + 标签y）
    X_rent = df_rent.drop('Price', axis=1)
    y_rent = df_rent['Price']
    X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
        X_rent, y_rent, test_size=0.2, random_state=111
    )
    print("租金数据划分完成：训练集{}条，测试集{}条".format(len(X_rent_train), len(X_rent_test)))

except Exception as e:
    print("执行出错：", e)

# ---------------------- 2. 特征筛选函数 ----------------------
def select_single_term_features(X_train, X_test, feature_names, y_train):
    """
    特征筛选流程：
    1. 筛选数值型特征（跳过非数值列）
    2. 过滤含缺失值的列（存在任何缺失值则跳过）
    3. 方差过滤 → 相关性分析 → VIF共线性检测 → Lasso正则化选择
    """
    # 1. 筛选数值型特征
    numeric_features = [col for col in feature_names if pd.api.types.is_numeric_dtype(X_train[col])]
    X_train = X_train[numeric_features]
    X_test = X_test[numeric_features]
    feature_names = numeric_features
    print(f"数值型特征保留：{len(numeric_features)}个")
    if not feature_names:
        print("警告：无有效数值型特征！")
        return X_train, X_test, []
    
    # 2. 过滤含缺失值的列（存在任何缺失值则跳过该列）
    missing_counts = X_train.isnull().sum()
    no_missing_features = [col for col in feature_names if missing_counts[col] == 0]
    X_train = X_train[no_missing_features]
    X_test = X_test[no_missing_features]
    feature_names = no_missing_features
    print(f"过滤含缺失值的列后保留：{len(no_missing_features)}个")
    if not feature_names:
        print("警告：所有数值型特征均含缺失值，已全部过滤！")
        return X_train, X_test, []
    
    # 3. 方差过滤（移除低方差特征）
    var_thresh = VarianceThreshold(threshold=0.01)
    X_train_var = var_thresh.fit_transform(X_train)
    var_mask = var_thresh.get_support()
    var_features = [f for i, f in enumerate(feature_names) if var_mask[i]]
    X_train_selected = pd.DataFrame(X_train_var, columns=var_features, index=X_train.index)
    X_test_selected = X_test[var_features]
    print(f"方差过滤后保留：{len(var_features)}个")
    
    # 4. 相关性分析（与目标变量）
    corr_df = pd.concat([X_train_selected, y_train], axis=1).corr()
    target_corr = corr_df[y_train.name].drop(y_train.name)
    corr_features = target_corr[abs(target_corr) > 0.1].index.tolist()
    X_train_selected = X_train_selected[corr_features]
    X_test_selected = X_test_selected[corr_features]
    print(f"相关性过滤后保留：{len(corr_features)}个")
    
    # 5. VIF共线性检测
    def calculate_vif(df):
        vif = pd.DataFrame()
        vif['特征'] = df.columns
        vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        return vif.sort_values('VIF')
    vif_features = corr_features.copy()
    while vif_features:
        vif_df = calculate_vif(X_train_selected[vif_features])
        max_vif = vif_df['VIF'].max()
        if max_vif > 10:
            drop_feature = vif_df.loc[vif_df['VIF'].idxmax(), '特征']
            vif_features.remove(drop_feature)
        else:
            break
    X_train_selected = X_train_selected[vif_features] if vif_features else X_train_selected
    X_test_selected = X_test_selected[vif_features] if vif_features else X_test_selected
    print(f"VIF过滤后保留：{len(vif_features)}个")
    
    # 6. Lasso正则化选择
    if not vif_features:
        print("警告：VIF过滤后无特征剩余！")
        return X_train_selected, X_test_selected, []
    lasso = Lasso(alpha=0.01, random_state=111)
    lasso.fit(X_train_selected, y_train)
    lasso_features = [f for i, f in enumerate(vif_features) if lasso.coef_[i] != 0]
    X_train_final = X_train_selected[lasso_features]
    X_test_final = X_test_selected[lasso_features]
    print(f"Lasso筛选后最终保留：{len(lasso_features)}个")
    print(f"最终特征列表：{lasso_features}\n")
    
    return X_train_final, X_test_final, lasso_features

# ---------------------- 3. 执行特征筛选（含强制保留指定特征） ----------------------
# 定义需要强制保留的特征
price_keep_features = [ '反馈情感_反馈_情感倾向_消极', '反馈情感_反馈_情感倾向_积极']
rent_keep_features = ['反馈情感_反馈_情感倾向_消极', '反馈情感_反馈_情感倾向_积极']

# 房价数据特征选择
print("===== 房价数据特征选择 =====")
# 提取强制保留的特征
X_price_train_keep = X_price_train[price_keep_features].copy()
X_price_test_keep = X_price_test[price_keep_features].copy()
# 处理剩余特征
X_price_train_other = X_price_train.drop(price_keep_features, axis=1, errors='ignore')
X_price_test_other = X_price_test.drop(price_keep_features, axis=1, errors='ignore')
X_price_train_other_selected, X_price_test_other_selected, _ = select_single_term_features(
    X_price_train_other, X_price_test_other, X_price_train_other.columns.tolist(), y_price_train
)
# 合并强制保留特征与筛选后剩余特征
X_price_train_selected = pd.concat([X_price_train_keep, X_price_train_other_selected], axis=1)
X_price_test_selected = pd.concat([X_price_test_keep, X_price_test_other_selected], axis=1)
price_selected_features = price_keep_features + X_price_train_other_selected.columns.tolist()
print(f"房价最终保留特征：{price_selected_features}\n")

# 租金数据特征选择
print("===== 租金数据特征选择 =====")
# 提取强制保留的特征
X_rent_train_keep = X_rent_train[rent_keep_features].copy()
X_rent_test_keep = X_rent_test[rent_keep_features].copy()
# 处理剩余特征
X_rent_train_other = X_rent_train.drop(rent_keep_features, axis=1, errors='ignore')
X_rent_test_other = X_rent_test.drop(rent_keep_features, axis=1, errors='ignore')
X_rent_train_other_selected, X_rent_test_other_selected, _ = select_single_term_features(
    X_rent_train_other, X_rent_test_other, X_rent_train_other.columns.tolist(), y_rent_train
)
# 合并强制保留特征与筛选后剩余特征
X_rent_train_selected = pd.concat([X_rent_train_keep, X_rent_train_other_selected], axis=1)
X_rent_test_selected = pd.concat([X_rent_test_keep, X_rent_test_other_selected], axis=1)
rent_selected_features = rent_keep_features + X_rent_train_other_selected.columns.tolist()
print(f"租金最终保留特征：{rent_selected_features}\n")

# ---------------------- 4. 保存特征数据 + 标签 ----------------------
# 保存房价相关数据（特征 + 标签）
X_price_train_selected.to_csv(f"{save_path}price_train_selected.csv", index=False)
X_price_test_selected.to_csv(f"{save_path}price_test_selected.csv", index=False)
y_price_train.to_csv(f"{save_path}y_price_train.csv", index=False, header=['Price'])  # 标签保存为单列，列名Price
y_price_test.to_csv(f"{save_path}y_price_test.csv", index=False, header=['Price'])

# 保存租金相关数据（特征 + 标签）
X_rent_train_selected.to_csv(f"{save_path}rent_train_selected.csv", index=False)
X_rent_test_selected.to_csv(f"{save_path}rent_test_selected.csv", index=False)
y_rent_train.to_csv(f"{save_path}y_rent_train.csv", index=False, header=['Price'])
y_rent_test.to_csv(f"{save_path}y_rent_test.csv", index=False, header=['Price'])

房价数据读取完成，共96048条
租金数据读取完成，共93365条
房价数据划分完成：训练集76838条，测试集19210条
租金数据划分完成：训练集74692条，测试集18673条
===== 房价数据特征选择 =====
数值型特征保留：439个
过滤含缺失值的列后保留：437个
方差过滤后保留：161个
相关性过滤后保留：31个
VIF过滤后保留：23个
Lasso筛选后最终保留：23个
最终特征列表：['套内面积', '容 积 率', '供热费', '简化户型_2室2厅1卫', '具体朝向_其他组合', '简化产权_商品房', '简化产权_商品房|已购公房|私产', '简化产权_商品房|私产', '环线_二至三环', '环线_五至六环', '环线_四至五环', '环线_nan', '建筑结构_混合结构', '交易权属_已购公房', '房屋用途_别墅', '房屋用途_商住两用', '房屋优势_地铁、房本满五年', '房屋优势_地铁、装修、房本满五年', '房屋优势_装修', '供暖_集中供暖', '总楼层', '实际楼层_底层', '实际楼层_顶层']

房价最终保留特征：['反馈情感_反馈_情感倾向_消极', '反馈情感_反馈_情感倾向_积极', '套内面积', '容 积 率', '供热费', '简化户型_2室2厅1卫', '具体朝向_其他组合', '简化产权_商品房', '简化产权_商品房|已购公房|私产', '简化产权_商品房|私产', '环线_二至三环', '环线_五至六环', '环线_四至五环', '环线_nan', '建筑结构_混合结构', '交易权属_已购公房', '房屋用途_别墅', '房屋用途_商住两用', '房屋优势_地铁、房本满五年', '房屋优势_地铁、装修、房本满五年', '房屋优势_装修', '供暖_集中供暖', '总楼层', '实际楼层_底层', '实际楼层_顶层']

===== 租金数据特征选择 =====
数值型特征保留：173个
过滤含缺失值的列后保留：171个
方差过滤后保留：120个
相关性过滤后保留：33个
VIF过滤后保留：23个
Lasso筛选后最终保留：23个
最终特征列表：['城市', '面积', '区县', '房屋总数', '供热费', '停车位', '停车费用', '租期_分数', '简化产权_商品房',

## 模型构建与评估

In [54]:
# ---------------------- 数据路径与加载 ----------------------
data_path = "/Users/guodehao/Documents/python/"

# 房价数据
X_price_train = pd.read_csv(f"{data_path}price_train_selected.csv")
X_price_test = pd.read_csv(f"{data_path}price_test_selected.csv")
y_price_train = pd.read_csv(f"{data_path}y_price_train.csv").squeeze()
y_price_test = pd.read_csv(f"{data_path}y_price_test.csv").squeeze()

# 租金数据
X_rent_train = pd.read_csv(f"{data_path}rent_train_selected.csv")
X_rent_test = pd.read_csv(f"{data_path}rent_test_selected.csv")
y_rent_train = pd.read_csv(f"{data_path}y_rent_train.csv").squeeze()
y_rent_test = pd.read_csv(f"{data_path}y_rent_test.csv").squeeze()


# ---------------------- 标准化函数 ----------------------
def standardize_features(X_train, X_test):
    """
    对训练集和测试集进行标准化（使用训练集的均值和标准差）
    避免数据泄露：测试集不能影响标准化参数
    """
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)  # 用训练集拟合scaler并转换
    X_test_scaled = scaler.transform(X_test)        # 用训练集的scaler转换测试集
    # 转换回DataFrame（保留列名，方便后续系数分析）
    return pd.DataFrame(X_train_scaled, columns=X_train.columns), pd.DataFrame(X_test_scaled, columns=X_test.columns)


# ---------------------- 多重共线性检查 ----------------------
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["特征"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values("VIF", ascending=False)


# ---------------------- 模型评估函数 ----------------------
def evaluate_linear_models(X_train, y_train, X_test, y_test, model_name, model):
    model.fit(X_train, y_train)
    
    y_range = y_train.max() - y_train.min()
    
    y_train_pred = model.predict(X_train)
    in_sample_mae = mean_absolute_error(y_train, y_train_pred)
    in_sample_rmae = in_sample_mae / y_range
    
    y_test_pred = model.predict(X_test)
    out_sample_mae = mean_absolute_error(y_test, y_test_pred)
    out_sample_rmae = out_sample_mae / y_range
    
    cv_scores = cross_val_score(
        model, X_train, y_train, cv=6, scoring='neg_mean_absolute_error'
    )
    cv_mae = -cv_scores.mean()
    cv_rmae = cv_mae / y_range
    
    return {
        "模型": model_name,
        "样本内MAE": round(in_sample_mae, 4),
        "样本内RMAE": round(in_sample_rmae, 4),
        "样本外MAE": round(out_sample_mae, 4),
        "样本外RMAE": round(out_sample_rmae, 4),
        "6折交叉验证MAE": round(cv_mae, 4),
        "6折交叉验证RMAE": round(cv_rmae, 4)
    }


# ---------------------- 系数对比函数 ----------------------
def print_coefficients(models, feature_names, model_names):
    coeff_df = pd.DataFrame()
    for model, name in zip(models, model_names):
        coeff_df[name] = model.coef_
    coeff_df.index = feature_names


# ---------------------- 房价模型----------------------
print("===== 房价数据处理 =====")
# 对房价特征进行标准化（关键修改）
X_price_train_scaled, X_price_test_scaled = standardize_features(X_price_train, X_price_test)

print("\n===== 房价模型评估结果 =====")
# 1. OLS回归（标准化后结果更合理，但OLS本身不需要标准化，这里为了对比统一处理）
ols_price = LinearRegression()
ols_price_result = evaluate_linear_models(
    X_price_train_scaled, y_price_train, X_price_test_scaled, y_price_test, 
    "OLS（房价）", ols_price
)

# 2. Lasso回归（标准化后效果更明显）
lasso_price = LassoCV(
    alphas=np.logspace(-4, 2, 100),
    cv=6,
    random_state=111
)
lasso_price_result = evaluate_linear_models(
    X_price_train_scaled, y_price_train, X_price_test_scaled, y_price_test, 
    "Lasso（房价）", lasso_price
)
print(f"Lasso（房价）最优alpha值：{lasso_price.alpha_:.4f}")

# 3. Ridge回归（同样需要标准化）
ridge_price = RidgeCV(
    alphas=np.logspace(-4, 2, 100),
    cv=6
)
ridge_price_result = evaluate_linear_models(
    X_price_train_scaled, y_price_train, X_price_test_scaled, y_price_test, 
    "Ridge（房价）", ridge_price
)
print(f"Ridge（房价）最优alpha值：{ridge_price.alpha_:.4f}")

price_results = pd.DataFrame([ols_price_result, lasso_price_result, ridge_price_result])
print(price_results)

print_coefficients(
    models=[ols_price, lasso_price, ridge_price],
    feature_names=X_price_train.columns,
    model_names=["OLS系数", "Lasso系数", "Ridge系数"]
)


# ---------------------- 租金模型（增加标准化步骤） ----------------------
print("\n===== 租金数据处理 =====")
# 对租金特征进行标准化（关键修改）
X_rent_train_scaled, X_rent_test_scaled = standardize_features(X_rent_train, X_rent_test)

print("\n===== 租金模型评估结果 =====")
# 1. OLS回归
ols_rent = LinearRegression()
ols_rent_result = evaluate_linear_models(
    X_rent_train_scaled, y_rent_train, X_rent_test_scaled, y_rent_test, 
    "OLS（租金）", ols_rent
)

# 2. Lasso回归
lasso_rent = LassoCV(
    alphas=np.logspace(-4, 2, 100),
    cv=6,
    random_state=111
)
lasso_rent_result = evaluate_linear_models(
    X_rent_train_scaled, y_rent_train, X_rent_test_scaled, y_rent_test, 
    "Lasso（租金）", lasso_rent
)
print(f"Lasso（租金）最优alpha值：{lasso_rent.alpha_:.4f}")

# 3. Ridge回归
ridge_rent = RidgeCV(
    alphas=np.logspace(-4, 2, 100),
    cv=6
)
ridge_rent_result = evaluate_linear_models(
    X_rent_train_scaled, y_rent_train, X_rent_test_scaled, y_rent_test, 
    "Ridge（租金）", ridge_rent
)
print(f"Ridge（租金）最优alpha值：{ridge_rent.alpha_:.4f}")

rent_results = pd.DataFrame([ols_rent_result, lasso_rent_result, ridge_rent_result])
print(rent_results)

print_coefficients(
    models=[ols_rent, lasso_rent, ridge_rent],
    feature_names=X_rent_train.columns,
    model_names=["OLS系数", "Lasso系数", "Ridge系数"]
)


# ---------------------- 保存预测结果（使用标准化后的测试集） ----------------------
y_price_test_pred = pd.DataFrame({
    "实际值": y_price_test,
    "OLS预测值": ols_price.predict(X_price_test_scaled),
    "Lasso预测值": lasso_price.predict(X_price_test_scaled),
    "Ridge预测值": ridge_price.predict(X_price_test_scaled)
})
y_price_test_pred.to_csv(f"{data_path}price_pred_results.csv", index=False)

y_rent_test_pred = pd.DataFrame({
    "实际值": y_rent_test,
    "OLS预测值": ols_rent.predict(X_rent_test_scaled),
    "Lasso预测值": lasso_rent.predict(X_rent_test_scaled),
    "Ridge预测值": ridge_rent.predict(X_rent_test_scaled)
})
y_rent_test_pred.to_csv(f"{data_path}rent_pred_results.csv", index=False)

===== 房价数据处理 =====

===== 房价模型评估结果 =====
Lasso（房价）最优alpha值：100.0000
Ridge（房价）最优alpha值：37.6494
          模型       样本内MAE  样本内RMAE       样本外MAE  样本外RMAE    6折交叉验证MAE  \
0    OLS（房价）  641706.6237   0.1213  638442.9524   0.1207  642018.6321   
1  Lasso（房价）  641718.4235   0.1213  638453.0916   0.1207  642029.1636   
2  Ridge（房价）  641720.7209   0.1213  638457.4633   0.1207  642042.1968   

   6折交叉验证RMAE  
0      0.1214  
1      0.1214  
2      0.1214  

===== 租金数据处理 =====

===== 租金模型评估结果 =====
Lasso（租金）最优alpha值：0.3275
Ridge（租金）最优alpha值：37.6494
          模型       样本内MAE  样本内RMAE       样本外MAE  样本外RMAE    6折交叉验证MAE  \
0    OLS（租金）  166631.6534   0.1164  166642.6972   0.1164  166705.2534   
1  Lasso（租金）  166631.6400   0.1164  166642.6959   0.1164  166705.2006   
2  Ridge（租金）  166630.6527   0.1164  166641.8400   0.1164  166704.1631   

   6折交叉验证RMAE  
0      0.1164  
1      0.1164  
2      0.1164  


## 模型优化与评估

In [55]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# ---------------------- 加载数据 ----------------------
save_path = "/Users/guodehao/Documents/python/"
# 房价数据
X_price_train = pd.read_csv(f"{save_path}price_train_selected.csv")
y_price_train = pd.read_csv(f"{save_path}y_price_train.csv").squeeze()
X_price_test = pd.read_csv(f"{save_path}price_test_selected.csv")
y_price_test = pd.read_csv(f"{save_path}y_price_test.csv").squeeze()

# 租金数据
X_rent_train = pd.read_csv(f"{save_path}rent_train_selected.csv")
y_rent_train = pd.read_csv(f"{save_path}y_rent_train.csv").squeeze()
X_rent_test = pd.read_csv(f"{save_path}rent_test_selected.csv")
y_rent_test = pd.read_csv(f"{save_path}y_rent_test.csv").squeeze()

# ---------------------- 目标变量对数变换（处理右偏分布） ----------------------
def log_transform(y):
    if y.min() <= 0:
        y = y + abs(y.min()) + 1e-6  # 避免非正值
    return np.log(y)

# 房价目标变量变换
y_price_train_log = log_transform(y_price_train)
y_price_test_log = log_transform(y_price_test)
y_price_original = y_price_train.copy()  # 保留原始尺度用于RMAE

# 租金目标变量变换
y_rent_train_log = log_transform(y_rent_train)
y_rent_test_log = log_transform(y_rent_test)
y_rent_original = y_rent_train.copy()  # 保留原始尺度用于RMAE

# ---------------------- 构建模型流水线 ----------------------
def build_pipeline(model):
    return Pipeline([
        ('scaler', StandardScaler()),  # 特征标准化
        ('regressor', model)           # 待调优的模型
    ])

# ---------------------- 超参数调优 ----------------------
def tune_model(X, y, model, param_grid, cv=6):
    pipeline = build_pipeline(model)
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X, y)
    return grid_search

# ---------------------- 模型与参数网格（房价） ----------------------
elasticnet_price = ElasticNet(random_state=111, max_iter=20000)
elasticnet_param_grid_price = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
    'regressor__l1_ratio': [0.2, 0.5, 0.8]
}

# 调优房价模型
print("调优房价ElasticNet模型...")
elasticnet_grid_price = tune_model(X_price_train, y_price_train_log, elasticnet_price, elasticnet_param_grid_price)
best_model_price = elasticnet_grid_price.best_estimator_

# ---------------------- 模型与参数网格（租金） ----------------------
elasticnet_rent = ElasticNet(random_state=111, max_iter=20000)
elasticnet_param_grid_rent = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
    'regressor__l1_ratio': [0.2, 0.5, 0.8]
}

# 调优租金模型
print("调优租金ElasticNet模型...")
elasticnet_grid_rent = tune_model(X_rent_train, y_rent_train_log, elasticnet_rent, elasticnet_param_grid_rent)
best_model_rent = elasticnet_grid_rent.best_estimator_

# ---------------------- 评估：计算RMAE（标准化相对误差） ----------------------
def calculate_rmae(y_true, y_pred, normalization='mean'):
    mae = mean_absolute_error(y_true, y_pred)
    if normalization == 'mean':
        return mae / np.mean(y_true)
    elif normalization == 'range':
        return mae / (np.max(y_true) - np.min(y_true))
    else:
        raise ValueError("normalization must be 'mean' or 'range'")

# ---------------------- 房价模型评估 ----------------------
# 预测并还原为原始尺度
y_price_pred_log = best_model_price.predict(X_price_test)
y_price_pred_original = np.exp(y_price_pred_log)
y_price_true_original = y_price_test  # 测试集原始真实值

# 样本内RMAE（房价训练集）
y_price_train_pred_log = best_model_price.predict(X_price_train)
y_price_train_pred_original = np.exp(y_price_train_pred_log)
price_train_rmae = calculate_rmae(y_price_original, y_price_train_pred_original, normalization='mean')

# 测试集RMAE（房价）
price_test_rmae = calculate_rmae(y_price_true_original, y_price_pred_original, normalization='mean')

# 交叉验证RMAE（房价）
y_price_cv_pred_log = cross_val_predict(best_model_price, X_price_train, y_price_train_log, cv=6)
y_price_cv_pred_original = np.exp(y_price_cv_pred_log)
price_cv_rmae = calculate_rmae(y_price_original, y_price_cv_pred_original, normalization='mean')

# ---------------------- 租金模型评估 ----------------------
# 预测并还原为原始尺度
y_rent_pred_log = best_model_rent.predict(X_rent_test)
y_rent_pred_original = np.exp(y_rent_pred_log)
y_rent_true_original = y_rent_test  # 测试集原始真实值

# 样本内RMAE（租金训练集）
y_rent_train_pred_log = best_model_rent.predict(X_rent_train)
y_rent_train_pred_original = np.exp(y_rent_train_pred_log)
rent_train_rmae = calculate_rmae(y_rent_original, y_rent_train_pred_original, normalization='mean')

# 测试集RMAE（租金）
rent_test_rmae = calculate_rmae(y_rent_true_original, y_rent_pred_original, normalization='mean')

# 交叉验证RMAE（租金）
y_rent_cv_pred_log = cross_val_predict(best_model_rent, X_rent_train, y_rent_train_log, cv=6)
y_rent_cv_pred_original = np.exp(y_rent_cv_pred_log)
rent_cv_rmae = calculate_rmae(y_rent_original, y_rent_cv_pred_original, normalization='mean')

# ---------------------- 输出结果 ----------------------
print("\n===== 房价模型结果 =====")
print(f"最佳参数：{elasticnet_grid_price.best_params_}")
print(f"样本内RMAE（均值标准化）：{price_train_rmae:.4f}")
print(f"测试集RMAE（均值标准化）：{price_test_rmae:.4f}")
print(f"6折交叉验证RMAE（均值标准化）：{price_cv_rmae:.4f}")

print("\n===== 租金模型结果 =====")
print(f"最佳参数：{elasticnet_grid_rent.best_params_}")
print(f"样本内RMAE（均值标准化）：{rent_train_rmae:.4f}")
print(f"测试集RMAE（均值标准化）：{rent_test_rmae:.4f}")
print(f"6折交叉验证RMAE（均值标准化）：{rent_cv_rmae:.4f}")

调优房价ElasticNet模型...
Fitting 6 folds for each of 15 candidates, totalling 90 fits
调优租金ElasticNet模型...
Fitting 6 folds for each of 15 candidates, totalling 90 fits

===== 房价模型结果 =====
最佳参数：{'regressor__alpha': 0.001, 'regressor__l1_ratio': 0.5}
样本内RMAE（均值标准化）：0.3715
测试集RMAE（均值标准化）：0.3700
6折交叉验证RMAE（均值标准化）：0.3717

===== 租金模型结果 =====
最佳参数：{'regressor__alpha': 0.001, 'regressor__l1_ratio': 0.2}
样本内RMAE（均值标准化）：0.3435
测试集RMAE（均值标准化）：0.3441
6折交叉验证RMAE（均值标准化）：0.3436


## 预测测试集

In [50]:

# ---------------------- 路径配置 ----------------------
save_path = "/Users/guodehao/Documents/python/"
test_price_path = "/Users/guodehao/Documents/test_price_preprocessed.csv"
test_rent_path = "/Users/guodehao/Documents/test_rent_preprocessed.csv"
output_price_pred = "/Users/guodehao/Documents/price_pred_result.csv"
output_rent_pred = "/Users/guodehao/Documents/rent_pred_result.csv"
output_merged_pred = "/Users/guodehao/Documents/merged_pred_result.csv"

# ---------------------- 辅助函数：目标变量还原 ----------------------
def exp_transform_y(y_log):
    return np.exp(y_log)

# ---------------------- 1. 加载最佳模型（基于训练阶段的最优参数） ----------------------
# 房价模型（使用最佳alpha）
X_price_train = pd.read_csv(f"{save_path}price_train_selected.csv")
price_features = X_price_train.columns.tolist()
y_price_train = pd.read_csv(f"{save_path}y_price_train.csv").squeeze()
# 训练时对目标变量的对数变换（与预测还原对应）
y_price_train_log = np.log(y_price_train + (1e-6 if y_price_train.min() <= 0 else 0))
# 构建最佳模型流水线（标准化+Lasso）
price_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=0.001, random_state=111, max_iter=20000))  # 最佳alpha
])
price_pipeline.fit(X_price_train, y_price_train_log)

# 租金模型（使用最佳alpha）
X_rent_train = pd.read_csv(f"{save_path}rent_train_selected.csv")
rent_features = X_rent_train.columns.tolist()
y_rent_train = pd.read_csv(f"{save_path}y_rent_train.csv").squeeze()
y_rent_train_log = np.log(y_rent_train + (1e-6 if y_rent_train.min() <= 0 else 0))
rent_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=0.01, random_state=111, max_iter=20000))  # 最佳alpha
])
rent_pipeline.fit(X_rent_train, y_rent_train_log)

# ---------------------- 2. 房价预测 ----------------------
df_test_price = pd.read_csv(test_price_path, low_memory=False)
test_price_ids = df_test_price['ID'].copy()
X_test_price = df_test_price.drop('ID', axis=1)

# 特征强制对齐
common_price_feats = [f for f in price_features if f in X_test_price.columns]
X_test_price = X_test_price[common_price_feats].copy()
for f in price_features:
    if f not in X_test_price.columns:
        X_test_price[f] = 0
X_test_price = X_test_price[price_features]

# 预测并还原为原始价格
price_pred_log = price_pipeline.predict(X_test_price)
price_pred = exp_transform_y(price_pred_log).round(2)
pd.DataFrame({"ID": test_price_ids, "Price": price_pred}).to_csv(output_price_pred, index=False)

# ---------------------- 3. 租金预测 ----------------------
df_test_rent = pd.read_csv(test_rent_path, low_memory=False)
test_rent_ids = df_test_rent['ID'].copy()
X_test_rent = df_test_rent.drop('ID', axis=1)

# 特征强制对齐
common_rent_feats = [f for f in rent_features if f in X_test_rent.columns]
X_test_rent = X_test_rent[common_rent_feats].copy()
for f in rent_features:
    if f not in X_test_rent.columns:
        X_test_rent[f] = 0
X_test_rent = X_test_rent[rent_features]

# 预测并还原
rent_pred_log = rent_pipeline.predict(X_test_rent)
rent_pred = exp_transform_y(rent_pred_log).round(2)
pd.DataFrame({"ID": test_rent_ids, "Price": rent_pred}).to_csv(output_rent_pred, index=False)

# ---------------------- 4. 合并结果 ----------------------
merged_pred_df = pd.concat([
    pd.read_csv(output_price_pred),
    pd.read_csv(output_rent_pred)
], ignore_index=True)
merged_pred_df.to_csv(output_merged_pred, index=False)

print(f"预测完成：\n房价结果：{output_price_pred}\n租金结果：{output_rent_pred}\n合并结果：{output_merged_pred}")

预测完成：
房价结果：/Users/guodehao/Documents/price_pred_result.csv
租金结果：/Users/guodehao/Documents/rent_pred_result.csv
合并结果：/Users/guodehao/Documents/merged_pred_result.csv
